In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, LSTM, LeakyReLU, Dropout
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import tensorflow as tf

In [ ]:
data = pd.read_csv("CSV directory")
data.head()

In [ ]:
power_electricity = data[['power_electricity']]
power_electricity_data = go.Scatter(x = data['Date_Time'], y = data['power_electricity'])
layout = go.Layout(title ='power_electricity time series', xaxis = dict(title='Date'), yaxis = dict(title='Power electricity'))

fig = go.Figure(data = [power_electricity_data], layout = layout)
fig.write_html('Scatter_Power_Electricity.html', auto_open = True)

In [ ]:
min_max_scaler = MinMaxScaler()
norm_data = min_max_scaler.fit_transform(power_electricity.values)
norm_data
norm_data_df = pd.DataFrame(norm_data)
norm_data_df.head()

In [ ]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []
    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size
    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        labels.append(dataset[i+target_size])
    return np.array(data), np.array(labels)

In [ ]:
past_history = 5
future_target = 0

TRAIN_SPLIT = int(len(norm_data) * 0.8)

x_train, y_train = univariate_data(norm_data,
                                   0,
                                   TRAIN_SPLIT,
                                   past_history,
                                   future_target)

x_test, y_test = univariate_data(norm_data,
                                 TRAIN_SPLIT,
                                 None,
                                 past_history,
                                 future_target)

In [ ]:
num_units = 64
learning_rate = 0.0001
activation_function = 'sigmoid'
adam = Adam(lr=learning_rate)
loss_function = 'mse'
batch_size = 5
num_epochs = 50

# Initialize the RNN
model = Sequential()
model.add(LSTM(units = num_units, activation=activation_function, input_shape=(None, 1)))
model.add(LeakyReLU(alpha=0.5))
model.add(Dropout(0.1))
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(optimizer=adam, loss=loss_function)

In [ ]:
history = model.fit(
    x_train,
    y_train,
    validation_split=0.1,
    batch_size=batch_size,
    epochs=num_epochs,
    shuffle=False
)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))
epochs_list = [*range(0, 50, 1)]

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x = epochs_list, y = loss, mode = 'lines', name = 'Loss'))
fig2.add_trace(go.Scatter(x = epochs_list, y = val_loss, mode = 'lines', name = 'Loss Val'))
fig2.update_layout(
    title="Power electricity Loss VS Loss Val",
    xaxis_title="Epochs",
    yaxis_title="Loss and Loss Val",
)
fig2.write_html('Power electricity Loss VS Los Val.html', auto_open = True)

In [ ]:
original = pd.DataFrame(min_max_scaler.inverse_transform(y_test))
predictions = pd.DataFrame(min_max_scaler.inverse_transform(model.predict(x_test)))

In [ ]:
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x = original.index, y = original[0], mode = 'lines', name = 'Test Data'))
fig3.add_trace(go.Scatter(x = predictions.index, y = predictions[0], mode = 'lines', name = 'Predictions'))
fig3.update_layout(
    title="Original VS Predictions",
    xaxis_title="Days",
    yaxis_title="Original and Predictions",
)
fig3.write_html('Original VS Predictions Power electricity.html', auto_open = True)